### Kernels and Gradients

In [2]:
%matplotlib inline
from scipy.spatial.distance import pdist, squareform
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import pairwise_kernels
from scipy.optimize import fmin_l_bfgs_b
from sklearn.datasets.samples_generator import make_blobs
from scipy.linalg import cholesky, cho_solve, solve_triangular

In [3]:
Z = np.array([[1,2], [3,4], [5,6]])

# RBF kernel

Squared exponential kernel (a.k.a RBF) with hyperparameter $l$:
$$ K(x_i, x_j) = \exp \Big(\frac{-|| x_i - x_j ||^2}{2l^2} \Big) $$
Kernel gradient is:
$$  \frac{d K(x_i, x_j)}{d l}  = \frac{||x_i - x_j||^2}{l^3}  K(x_i, x_j) $$

In [4]:
theta = np.array([2,2])

In [5]:
pdist(Z, 'euclidean')

array([2.82842712, 5.65685425, 2.82842712])

In [6]:
np.linalg.norm(Z[1] - Z[2])

2.8284271247461903

In [7]:
(1/theta**2)

array([0.25, 0.25])

In [8]:
%%timeit -n 1000
pdist(Z, 'minkowski', p=2, w=(1/theta**2))

151 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
Z

array([[1, 2],
       [3, 4],
       [5, 6]])

In [39]:
def squared_exp_kernel(X, theta):
    pairwise_dists = squareform(pdist(X, 'euclidean'))**2
    K = np.exp(-pairwise_dists/ (2 * theta[0] ** 2))
    K_gradient = np.multiply(pairwise_dists/(theta[0]**3), K)
    return K, np.expand_dims(K_gradient, axis=2)

In [40]:
def squared_exp_kernel_func(a, b, theta_):
    # theta includes number of parameters
    squared_dist = np.linalg.norm(a-b)**2
    k = np.exp(-squared_dist/(2*theta_[0]**2))
    return k

In [41]:
#%%timeit -n 1000
K2, K2_gradient = squared_exp_kernel(Z, [2])

In [42]:
K2

array([[1.        , 0.36787944, 0.01831564],
       [0.36787944, 1.        , 0.36787944],
       [0.01831564, 0.36787944, 1.        ]])

In [43]:
K2_gradient.shape

(3, 3, 1)

In [44]:
K2_gradient

array([[[0.        ],
        [0.36787944],
        [0.07326256]],

       [[0.36787944],
        [0.        ],
        [0.36787944]],

       [[0.07326256],
        [0.36787944],
        [0.        ]]])

In [15]:
from sklearn.gaussian_process.kernels import RBF
my_kernel = RBF(2.0)
A, B = my_kernel(Z, eval_gradient=True)

In [16]:
A

array([[1.        , 0.36787944, 0.01831564],
       [0.36787944, 1.        , 0.36787944],
       [0.01831564, 0.36787944, 1.        ]])

In [47]:
B

array([[[0.        ],
        [0.73575888],
        [0.14652511]],

       [[0.73575888],
        [0.        ],
        [0.73575888]],

       [[0.14652511],
        [0.73575888],
        [0.        ]]])

# ARD Kernel

Automatic Relevance Determination (ARD) squared exponential kernel:
$$ K(x_i, x_j) = \exp \Big( \sum_{d=1}^{D} \frac{-|| x_i - x_j ||^2}{2l_d^2} \Big)  $$
where $l_d$ is scaling hyperparameter for each dimension

Kernel gradient is:
$$  \frac{d K(x_i, x_j)}{d l_d}  = \frac{||x_{id} - x_{jd}||^2}{l_d^3}  K(x_i, x_j) $$

In [29]:
Z

array([[1, 2],
       [3, 4],
       [5, 6]])

In [30]:
Z[1]

array([3, 4])

In [33]:
wminkowski(Z[0], Z[1], 2, 1/theta)**2

2.0000000000000004

In [57]:
theta = np.array([2,3])

In [34]:
( ( (Z[0][0] - Z[2][0]) / theta[0] )**2 + ( (Z[0][1] - Z[2][1]) / theta[1] ) ) ** (0.5)

1.4142135623730951

In [35]:
Z[:,0]

array([1, 3, 5])

In [104]:
def ard_squared_exp_kernel(X, theta_):
    #%%timeit -n 1000
    pairwise_dists = squareform(pdist(X, 'minkowski', p=2, w=(1/theta_**2)))**2
    K = np.exp(-pairwise_dists/2)
    
    # calculate gradient
    temp_X = np.zeros([X.shape[0], 2])
    K_gradients = []
    for dim in range(X.shape[1]):
        temp_X[:,dim] = Z[:,dim].copy()
        temp_K = squareform( pdist(temp_X, 'sqeuclidean')/theta[dim]**3 )
        K_gradients.append(np.multiply(K, temp_K))
    return K, np.array(K_gradients)

In [115]:
%%timeit -n 1000
K3, K3_gradient = ard_squared_exp_kernel(Z, theta)

179 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [113]:
K3

array([[1.        , 0.48567179, 0.055638  ],
       [0.48567179, 1.        , 0.48567179],
       [0.055638  , 0.48567179, 1.        ]])

In [112]:
K3_gradient

array([[[0.        , 0.24283589, 0.111276  ],
        [0.24283589, 0.        , 0.24283589],
        [0.111276  , 0.24283589, 0.        ]],

       [[0.        , 0.14390275, 0.06594133],
        [0.14390275, 0.        , 0.14390275],
        [0.06594133, 0.14390275, 0.        ]]])

In [56]:
#%%timeit -n 1000
pdist(Z, 'minkowski', p=2, w=(1/theta**2))**2

array([1.41421356, 2.82842712, 1.41421356])

In [79]:
Z1 = np.zeros([Z.shape[0], 2])

In [84]:
Z1[:,0] = Z[:,0]

In [85]:
Z1

array([[1., 0.],
       [3., 0.],
       [5., 0.]])

In [95]:
pdist(Z1, 'sqeuclidean')/theta[0]**3

array([0.5, 2. , 0.5])

In [88]:
np.exp(-1/2-2/9)

0.4856717852477124

In [135]:
def ard_exp_kernel(a, b, theta_):
    # theta includes number of parameters
    squared_dist = (a-b)**2
    squared_dist = np.dot(squared_dist, 1/theta_**2)
    return np.exp(-squared_dist/2)

In [160]:
def ard_exp_kernel_gradient(a,b, theta_):
    squared_dist = (a-b)**2
    #print(squared_dist)
    return squared_dist * (1/theta_**3)

In [189]:
def ard_exp_kernel2(X, theta_):
    K = pairwise_kernels(Z, metric=ard_exp_kernel,  theta_= theta)
    K_gradients = []
    for dim in range(X.shape[1]):
        k_gradient_temp = np.multiply( pairwise_kernels(Z[:,dim].reshape(-1,1), metric=ard_exp_kernel_gradient,  theta_= theta[dim]), K)
        K_gradients.append(k_gradient_temp)  
    return K, np.array(K_gradients)

In [190]:
K = pairwise_kernels(Z, metric=ard_exp_kernel,  theta_= theta)

In [191]:
%%timeit -n 1000
K_gradient1 = np.multiply( pairwise_kernels(Z[:,0].reshape(-1,1), metric=ard_exp_kernel_gradient,  theta_= theta[0]), K)
K_gradient2 = np.multiply( pairwise_kernels(Z[:,1].reshape(-1,1), metric=ard_exp_kernel_gradient,  theta_= theta[1]), K)

164 µs ± 9.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [197]:
%%timeit -n 1000
K, K_gradient = ard_exp_kernel2(Z, theta)

195 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [193]:
K_gradient

array([[[0.        , 0.24283589, 0.111276  ],
        [0.24283589, 0.        , 0.24283589],
        [0.111276  , 0.24283589, 0.        ]],

       [[0.        , 0.07195138, 0.03297067],
        [0.07195138, 0.        , 0.07195138],
        [0.03297067, 0.07195138, 0.        ]]])

In [194]:
K

array([[1.        , 0.48567179, 0.055638  ],
       [0.48567179, 1.        , 0.48567179],
       [0.055638  , 0.48567179, 1.        ]])

Recursive ARD kernel:
$$ K(x_i, x_j) = \exp \Big( \sum_{d=1}^{D} \frac{-|| x_i - x_j ||^2}{2l_d^2} \Big) \exp \Big( \frac{K(x_i, x_j)_{pre} - 1}{\rho^2}  \Big)  $$


In [241]:
# initiliaze
theta=np.array([2, 3])
rho = 2

K_pre, K_pre_gradients  = ard_squared_exp_kernel(Z, theta)

In [251]:
np.append(theta, rho)

array([2, 3, 2])

In [233]:
K_pre_gradients = np.array([np.zeros(Z.shape[0]), np.zeros(Z.shape[0]), np.zeros(Z.shape[0])])

In [234]:
K_pre_gradients

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [236]:
def recursive_ard(X, theta_):
    rho  = theta_[-1]
    #print(np.eye(X.shape[0] ))
    theta_current = theta_[:-1]
    K_ard, K_gradients_ = ard_squared_exp_kernel(X, theta_current)
    K_state = np.exp( (K_pre - 1)/rho**2 )
    K = np.multiply(K_ard, K_state)
    return K

In [237]:
recursive_ard(Z, theta)

array([[1.        , 0.37824157, 0.04333092],
       [0.37824157, 1.        , 0.37824157],
       [0.04333092, 0.37824157, 1.        ]])